In [2]:
import bs4
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import nltk
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from textblob import TextBlob
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Phase 1

In [3]:
df = pd.read_csv('Flipkart - Redgear Gamepad.csv')

In [10]:
df.head(10)

,Unnamed: 0,Name,Date,Rating,Images,Likes,Dislikes,Review Title,Review Text,location
0,0,soumyadeep sharma,"Feb, 2019",5,5,518,78,Highly recommended,great product ... love it ... my favorite game...,North Twenty Four Parganas District
1,1,Krishna Rao,"Aug, 2016",5,0,106,15,Coolest piece ever had,"Very good product, must have for gamers.",Bangalore
2,2,Chanchal Pan,"Dec, 2018",5,1,44,4,Great product,best,Burdwan
3,3,abhishek kumar,"Jun, 2016",5,0,248,49,LIKE XBOX 360,At first time i got a defective piece it's rec...,Patna
4,4,YUSUF RASOOL KHAN,"Jul, 2021",5,1,27,2,Super!,"Very nice game pet, runs very smooth thank you...",Nagpur
5,5,Reena Rajput,"Feb, 2019",5,1,12,0,Super!,Awesome 👌,Jammu
6,6,Flipkart Customer,11 months ago,5,1,18,1,Terrific,Amazing product I love it vibration is very go...,Rohtak
7,7,Flipkart Customer,"Oct, 2021",5,0,17,1,Excellent,Yes. It works with Android TV and I know that ...,New Delhi
8,8,taha Adnan,"Dec, 2018",5,1,117,26,Terrific,Better than many other pricy gamepad..... Grea...,Warangal
9,9,Nilesh Talekar,"May, 2020",5,1,17,1,Mind-blowing purchase,i've been using this controller since 2 yearsi...,Aurangabad


In [11]:
# Drop the column 'Unnamed: 0'
df.drop("Unnamed: 0", axis=1, inplace=True)

In [12]:
df.describe()

,Rating,Images,Likes,Dislikes
count,2200.000000,2200.000000,2200.000000,2200.000000
mean,4.350455,0.123636,1.325455,0.417273
std,1.165441,0.489253,18.813812,4.824380
min,1.000000,0.000000,0.000000,0.000000
25%,4.000000,0.000000,0.000000,0.000000
50%,5.000000,0.000000,0.000000,0.000000
75%,5.000000,0.000000,0.000000,0.000000
max,5.000000,5.000000,622.000000,184.000000


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          2200 non-null   object
 1   Date          2200 non-null   object
 2   Rating        2200 non-null   int64 
 3   Images        2200 non-null   int64 
 4   Likes         2200 non-null   int64 
 5   Dislikes      2200 non-null   int64 
 6   Review Title  2200 non-null   object
 7   Review Text   2200 non-null   object
 8   location      2200 non-null   object
dtypes: int64(4), object(5)
memory usage: 154.8+ KB


In [14]:
df.Rating.value_counts()

Rating
5    1472
4     404
1     165
3     112
2      47
Name: count, dtype: int64

In [15]:
#Preprocessing

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc) #remove any characters that are not alphanumeric or whitespace
    doc = doc.lower() #lowercase
    doc = doc.strip() #Removes leading and trailing whitespaces
    tokens = nltk.word_tokenize(doc) #Tokenizes the document into individual words
    filtered_tokens = [token for token in tokens if token not in stop_words] #Filters out stopwords
    doc = ' '.join(filtered_tokens) #Reconstructs the document from the filtered tokens by joining them back into a single string separated by spaces
    return doc

normalize_corpus = np.vectorize(normalize_document) #Vectorizes the normalize_document function

norm_corpus = normalize_corpus(list(df['Review Text'])) # applying function to dataframe, producing a normalized corpus of documents
norm_corpus

array(['great product love favorite gamepad beautiful smooth',
       'good product must gamers', 'best', ...,
       'wireless game pad awesome supports xinput backlights also great definitely love',
       'good product smooth easy use 100 value money', 'good'],
      dtype='<U383')

In [16]:
#convert a collection of text documents (norm_corpus) into a matrix of token counts

cv = CountVectorizer(ngram_range=(1, 2), min_df=20, max_df=0.8) #both single words and pairs of consecutive words. Tokens that occur in fewer than 20 documents will be ignored. Tokens that occur in more than 80% of the documents will be ignored.
cv_matrix = cv.fit_transform(norm_corpus) #transforms the corpus into a document-term matrix. The value at the intersection of a row and column represents the count of how many times that word or n-gram appears in the corresponding document.
cv_matrix.shape #(n_documents, n_features). n_documents represents the number of documents in the corpus, and n_features represents the number of unique words and n-grams in the vocabulary.

(2200, 189)

In [17]:
sentiments = []
for text in norm_corpus:
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity
    if sentiment_score > 0:
        sentiments.append('Positive')
    elif sentiment_score < 0:
        sentiments.append('Negative')
    else:
        sentiments.append('Neutral')

# Add sentiment labels to DataFrame
df['Sentiment'] = sentiments

# Display the DataFrame with sentiment labels
print(df[['Review Text', 'Sentiment']])

                                            Review Text Sentiment
0     great product ... love it ... my favorite game...  Positive
1              Very good product, must have for gamers.  Positive
2                                                  best  Positive
3     At first time i got a defective piece it's rec...  Positive
4     Very nice game pet, runs very smooth thank you...  Positive
...                                                 ...       ...
2195             Best gamepad next to Xbox controller .  Positive
2196                       Nice product at this price..  Positive
2197  This wireless game pad is awesome. It supports...  Positive
2198  Good product. Very smooth & easy to use. 100% ...  Positive
2199                                               Good  Positive

[2200 rows x 2 columns]


In [24]:
df.head(30)

,Name,Date,Rating,Images,Likes,Dislikes,Review Title,Review Text,location,Sentiment
0,soumyadeep sharma,"Feb, 2019",5,5,518,78,Highly recommended,great product ... love it ... my favorite game...,North Twenty Four Parganas District,Positive
1,Krishna Rao,"Aug, 2016",5,0,106,15,Coolest piece ever had,"Very good product, must have for gamers.",Bangalore,Positive
2,Chanchal Pan,"Dec, 2018",5,1,44,4,Great product,best,Burdwan,Positive
3,abhishek kumar,"Jun, 2016",5,0,248,49,LIKE XBOX 360,At first time i got a defective piece it's rec...,Patna,Positive
4,YUSUF RASOOL KHAN,"Jul, 2021",5,1,27,2,Super!,"Very nice game pet, runs very smooth thank you...",Nagpur,Positive
5,Reena Rajput,"Feb, 2019",5,1,12,0,Super!,Awesome 👌,Jammu,Positive
6,Flipkart Customer,11 months ago,5,1,18,1,Terrific,Amazing product I love it vibration is very go...,Rohtak,Positive
7,Flipkart Customer,"Oct, 2021",5,0,17,1,Excellent,Yes. It works with Android TV and I know that ...,New Delhi,Positive
8,taha Adnan,"Dec, 2018",5,1,117,26,Terrific,Better than many other pricy gamepad..... Grea...,Warangal,Positive
9,Nilesh Talekar,"May, 2020",5,1,17,1,Mind-blowing purchase,i've been using this controller since 2 yearsi...,Aurangabad,Positive


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          2200 non-null   object
 1   Date          2200 non-null   object
 2   Rating        2200 non-null   int64 
 3   Images        2200 non-null   int64 
 4   Likes         2200 non-null   int64 
 5   Dislikes      2200 non-null   int64 
 6   Review Title  2200 non-null   object
 7   Review Text   2200 non-null   object
 8   location      2200 non-null   object
 9   Sentiment     2200 non-null   object
dtypes: int64(4), object(6)
memory usage: 172.0+ KB


In [26]:
sentiment_counts = df['Sentiment'].value_counts()
sentiment_counts

Sentiment
Positive    1838
Neutral      247
Negative     115
Name: count, dtype: int64

**Positive Sentiment (1838 reviews):**

High positive sentiment indicates that a significant portion of customers are satisfied with the product.
Businesses can use positive reviews for marketing purposes, such as testimonials on their website or social media channels.
It suggests that the product is meeting customer expectations and may lead to repeat purchases and positive word-of-mouth recommendations.

**Neutral Sentiment (247 reviews):**

Neutral sentiment could indicate a mixed reception from customers.
Businesses should investigate further to understand the reasons behind the neutral sentiment. It could be due to minor issues or areas where the product can be improved.
Neutral sentiment reviews provide an opportunity for businesses to gather constructive feedback and make necessary adjustments to enhance customer satisfaction.

**Negative Sentiment (115 reviews):**

Negative sentiment reviews highlight areas of dissatisfaction or issues with the product.
Businesses should prioritize addressing the concerns raised in negative reviews to improve customer experience and prevent further negative feedback.
Negative reviews can also be an opportunity for businesses to demonstrate their commitment to customer satisfaction by addressing issues promptly and transparently.

In [27]:
# Filter positive sentiment reviews
positive_reviews_df = df[df['Sentiment'] == 'Positive']

In [28]:
# Tokenize and preprocess positive sentiment reviews
positive_norm_corpus = normalize_corpus(positive_reviews_df['Review Text'])

In [29]:
# Vectorize text using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000, max_df=0.8, min_df=20)
tfidf_matrix = tfidf_vectorizer.fit_transform(positive_norm_corpus)

In [30]:
# Apply Latent Dirichlet Allocation (LDA) for topic modeling
num_topics = 10  # You can adjust the number of topics as needed
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_topic_matrix = lda_model.fit_transform(tfidf_matrix)

In [18]:
# Display topics and associated top words
feature_names = tfidf_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda_model.components_):
    print(f"Topic {topic_idx+1}:")
    top_words_idx = topic.argsort()[:-10 - 1:-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(top_words)

Topic 1:
['amazing', 'android', 'tv', 'game', 'works', 'product', 'pad', 'games', 'bought', 'awesome']
Topic 2:
['nice', 'product', 'love', 'gamepad', 'little', 'must', 'bit', 'cheap', 'buy', 'price']
Topic 3:
['best', 'controller', 'gamepad', 'price', 'range', 'budget', 'ever', 'pc', 'go', 'gamers']
Topic 4:
['good', 'battery', 'better', 'playing', 'life', 'gamepad', 'using', 'controller', 'feel', 'months']
Topic 5:
['worth', 'money', 'product', 'delivery', 'good', 'flipkart', 'value', 'happy', 'buy', 'dont']
Topic 6:
['one', 'work', 'good', 'satisfied', 'time', 'response', 'get', 'first', 'controller', 'cant']
Topic 7:
['great', 'superb', 'gaming', 'experience', 'product', 'vibration', 'controller', 'battery', 'backup', 'good']
Topic 8:
['good', 'product', 'awesome', 'price', 'loved', 'gamepad', 'really', 'range', 'go', 'got']
Topic 9:
['excellent', 'quality', 'super', 'good', 'working', 'perfect', 'product', 'build', 'fine', 'recommend']
Topic 10:
['smooth', 'easy', 'good', 'thanks'

**Product Performance and Functionality (Topics 1, 4, 7, 10):**

Customers appreciate the performance of the gamepad, including its compatibility with various devices (Android TV, PC), responsiveness, smoothness, and ease of use.
Implication: Businesses should focus on maintaining or improving the product's performance and functionality to meet customer expectations consistently. This may involve optimizing compatibility, enhancing battery life, and ensuring smooth gameplay experience.

**Value for Money (Topics 2, 5, 8):**

Positive reviews mention the product's affordability, value for money, and satisfaction with the purchase.
Implication: Businesses can emphasize the product's competitive pricing and value proposition in marketing campaigns to attract price-conscious customers. Offering discounts, promotions, or bundle deals may further enhance the perceived value of the product.

**Quality and Durability (Topics 3, 9):**

Customers praise the gamepad's build quality, durability, and overall performance, considering it among the best in its price range.
Implication: Businesses should prioritize maintaining product quality and durability standards to sustain positive customer sentiment and loyalty. Investing in high-quality materials, rigorous testing, and continuous improvement processes can contribute to long-term customer satisfaction.

**Customer Experience and Service (Topics 6):**

Positive reviews highlight satisfactory customer experiences, including prompt delivery, responsive customer service, and overall satisfaction with the purchase.
Implication: Businesses should prioritize providing excellent customer service, ensuring timely delivery, and addressing any issues or concerns promptly. Positive customer experiences contribute to brand loyalty and positive word-of-mouth referrals, driving future sales and customer retention.

**Feedback for Improvement (All Topics):**

While positive reviews generally reflect satisfaction with the product, they may also contain suggestions for improvement or feature requests.
Implication: Businesses should actively monitor and analyze customer feedback, both positive and negative, to identify areas for product enhancement or new feature development. Incorporating customer feedback into product development processes demonstrates a commitment to continuous improvement and customer-centric innovation.

In [31]:
# Perform KMeans clustering
num_clusters = 3  # You can adjust the number of clusters as needed
kmeans_model = KMeans(n_clusters=num_clusters, random_state=42)
kmeans_clusters = kmeans_model.fit_predict(tfidf_matrix)

/Users/bijitgoswami/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [32]:
# Add cluster labels to the DataFrame
positive_reviews_df['Cluster'] = kmeans_clusters

/var/folders/9w/pgl8trgx1c5bwkrmm5hd5ktw0000gn/T/ipykernel_16247/3589983744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_reviews_df['Cluster'] = kmeans_clusters


In [33]:
# Display the DataFrame with cluster labels
print(positive_reviews_df[['Review Text', 'Cluster']])

                                            Review Text  Cluster
0     great product ... love it ... my favorite game...        1
1              Very good product, must have for gamers.        1
2                                                  best        1
3     At first time i got a defective piece it's rec...        1
4     Very nice game pet, runs very smooth thank you...        2
...                                                 ...      ...
2195             Best gamepad next to Xbox controller .        1
2196                       Nice product at this price..        2
2197  This wireless game pad is awesome. It supports...        1
2198  Good product. Very smooth & easy to use. 100% ...        1
2199                                               Good        0

[1838 rows x 2 columns]


In [34]:
Cluster_counts = positive_reviews_df['Cluster'].value_counts()
Cluster_counts

Cluster
1    1397
0     257
2     184
Name: count, dtype: int64

**Understanding Customer Segments:**

The clustering results indicate that there are at least three distinct segments of customers who perceive the product positively.
Implication: Businesses should conduct further analysis to understand the characteristics and preferences of each customer segment. This information can help tailor marketing strategies, product features, and customer support to better meet the needs of each segment.

**Identifying Key Features or Benefits:**

Each cluster may represent customers who prioritize different features or benefits of the product.
Implication: By analyzing the characteristics of each cluster and the most common terms or topics within each cluster, businesses can identify key features or benefits that resonate most strongly with different customer segments. This insight can guide product development efforts, marketing messaging, and sales strategies.

**Personalized Marketing and Communication:**

Tailoring marketing messages and communication channels to specific customer segments can enhance engagement and conversion rates.
Implication: Businesses can create personalized marketing campaigns, advertisements, and promotions that highlight the features or benefits most relevant to each customer segment. This targeted approach can increase the effectiveness of marketing efforts and strengthen brand affinity among different customer groups.

**Product Customization or Bundling:**

Offering customizable options or bundled packages that cater to the preferences of different customer segments can increase customer satisfaction and sales.
Implication: Businesses can explore the possibility of offering product variations or bundled packages that align with the preferences of each customer segment. This approach allows customers to choose the features or benefits that matter most to them, enhancing perceived value and satisfaction with the product.

**Continuous Improvement and Innovation:**

Regularly monitoring customer feedback and clustering results can provide valuable insights for product improvement and innovation.
Implication: Businesses should establish mechanisms for collecting and analyzing customer feedback continuously. By identifying emerging trends, unmet needs, or areas of improvement within each customer segment, businesses can prioritize product enhancements and innovations to stay ahead of competitors and maintain customer satisfaction.

In [35]:
# Filter positive sentiment reviews
negative_reviews_df = df[df['Sentiment'] == 'Negative']

In [36]:
# Tokenize and preprocess positive sentiment reviews
negative_norm_corpus = normalize_corpus(negative_reviews_df['Review Text'])

In [37]:
# Vectorize text using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000, max_df=0.8, min_df=20)
tfidf_matrix = tfidf_vectorizer.fit_transform(negative_norm_corpus)

In [38]:
# Apply Latent Dirichlet Allocation (LDA) for topic modeling
num_topics = 3  # You can adjust the number of topics as needed
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_topic_matrix = lda_model.fit_transform(tfidf_matrix)

In [39]:
# Display topics and associated top words
feature_names = tfidf_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda_model.components_):
    print(f"Topic {topic_idx+1}:")
    top_words_idx = topic.argsort()[:-10 - 1:-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(top_words)

Topic 1:
['bad', 'product', 'game']
Topic 2:
['game', 'product', 'bad']
Topic 3:
['product', 'bad', 'game']


**Topic 1:**

Terms: 'bad', 'product', 'game'
Interpretation: This topic seems to focus on negative sentiments regarding the product quality or performance in the context of gaming.

**Topic 2:**

Terms: 'game', 'product', 'bad'
Interpretation: Similar to Topic 1, this topic also highlights dissatisfaction with the product's gaming-related aspects, such as performance or functionality.

**Topic 3:**

Terms: 'product', 'bad', 'game'
Interpretation: This topic reiterates the sentiment expressed in the previous topics, emphasizing negative feedback on the product's gaming-related features or overall quality.
Given the limited variation and similarity across topics, it appears that negative reviews predominantly focus on aspects related to the product's performance or quality in the context of gaming. This consistency in topics suggests a recurring issue or dissatisfaction among customers regarding these specific aspects of the product.


**Quality Improvement:**

The consistency of negative feedback regarding the product's performance or quality in gaming highlights the need for quality improvement.
Businesses should prioritize addressing the identified issues or shortcomings to enhance the product's functionality, reliability, and overall quality.

**Customer Support and Issue Resolution:**

Prompt and effective customer support is essential for addressing the concerns raised in negative reviews.
Businesses should establish mechanisms for customers to report issues and provide timely resolutions or replacements to mitigate dissatisfaction and rebuild trust.

**Product Development and Testing:**

Insights from negative reviews can inform future product development and testing processes.
Businesses should integrate customer feedback into product development cycles, conducting thorough testing and quality assurance to identify and rectify potential issues before product release.

**Communication and Transparency:**

Transparent communication about product improvements and resolutions can help manage customer expectations and mitigate negative sentiment.
Businesses should communicate openly with customers regarding identified issues, planned improvements, and resolutions to demonstrate their commitment to customer satisfaction.

**Competitive Analysis:**

Analyzing negative feedback in comparison to competitors' products can provide insights into areas of competitive advantage or weakness.
Businesses should benchmark their product against competitors' offerings, identifying opportunities for differentiation and improvement to remain competitive in the market.

In [40]:
# Perform KMeans clustering
num_clusters = 3  # You can adjust the number of clusters as needed
kmeans_model = KMeans(n_clusters=num_clusters, random_state=42)
kmeans_clusters = kmeans_model.fit_predict(tfidf_matrix)

/Users/bijitgoswami/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [41]:
# Add cluster labels to the DataFrame
negative_reviews_df['Cluster'] = kmeans_clusters

/var/folders/9w/pgl8trgx1c5bwkrmm5hd5ktw0000gn/T/ipykernel_16247/2315681470.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_reviews_df['Cluster'] = kmeans_clusters


In [42]:
# Display the DataFrame with cluster labels
print(negative_reviews_df[['Review Text', 'Cluster']])

                                            Review Text  Cluster
25    I don't know why people send used and old prod...        0
31    The game pad is average quality product and it...        1
84    This game controller is my first ever game con...        1
93                                  i wanna return it'!        0
94    joystick not working just after 1 month of usa...        0
...                                                 ...      ...
2066                         Need practice to play game        1
2107  Im not disappointed Wid dis its quality and re...        0
2110                         worst game pad don't buy..        1
2120                                           Not bad.        0
2123                   Late delivey  but awsome product        2

[115 rows x 2 columns]


In [43]:
Cluster_counts = negative_reviews_df['Cluster'].value_counts()
Cluster_counts

Cluster
0    73
2    22
1    20
Name: count, dtype: int64

**Identifying Common Issues:**

Analyzing the content and characteristics of each cluster can help identify common issues or themes that contribute to negative sentiment among customers.
Businesses should conduct further analysis to understand the underlying reasons behind the clustering and address the root causes of dissatisfaction.

**Prioritizing Resolution Efforts:**

Prioritizing resolution efforts based on the size and severity of each cluster can help businesses allocate resources effectively.
Addressing issues identified in the largest cluster (Cluster 0) first may have the most significant impact on mitigating negative sentiment and improving overall customer satisfaction.

**Tailoring Solutions to Cluster Needs:**

Tailoring solutions and improvements to address the specific needs and concerns of each cluster can enhance the effectiveness of resolution efforts.
Businesses should analyze the content and sentiment within each cluster to develop targeted strategies for addressing common issues and improving customer experiences.

**Continuous Monitoring and Improvement:**

Continuous monitoring of customer feedback and sentiment, including negative reviews, is essential for identifying emerging issues and trends.
Businesses should establish mechanisms for collecting and analyzing customer feedback regularly, allowing them to adapt and respond promptly to changing customer needs and preferences.

**Communication and Transparency:**

Transparent communication about identified issues, planned resolutions, and improvements can help manage customer expectations and rebuild trust.
Businesses should communicate openly with customers affected by negative experiences, providing updates and assurances regarding resolution efforts and improvements.

# Phase 2

In [44]:
# Function to parse date strings and convert to datetime objects
def parse_date(date_str):
    try:
        # Try to parse as a standard date format
        return pd.to_datetime(date_str)
    except ValueError:
        # If parsing as a standard date format fails, try to parse as "X months ago"
        try:
            months_ago = int(date_str.split()[0])
            return datetime.now() - relativedelta(months=months_ago)
        except (ValueError, IndexError):
            # If both parsing attempts fail, return NaT (Not a Time)
            return pd.NaT

# Apply the parse_date function to the 'Date' column
df['Date'] = df['Date'].apply(parse_date)

# Sort the DataFrame by 'Date' in descending order
df_sorted = df.sort_values(by='Date', ascending=False)

# Calculate the index for the 25th percentile
twentyfifth_percentile_index = int(0.25 * len(df_sorted))

# Select the most recent 25% of the reviews
most_recent_reviews = df_sorted.iloc[:twentyfifth_percentile_index]

In [45]:
most_recent_reviews

,Name,Date,Rating,Images,Likes,Dislikes,Review Title,Review Text,location,Sentiment
410,Flipkart Customer,2024-02-09 15:46:18.193739,5,0,0,0,Classy product,Best product for everyone. 👍 👌,Buxar,Positive
409,Priyabrata Garanayak,2024-02-09 15:46:18.193668,5,0,0,0,Fabulous!,"Nice, Battery backup is upto 8hr.....",Tipo,Positive
408,Yogesh Sharma,2024-02-09 15:46:18.193598,4,0,0,0,Good quality product,Nice product value for money,Udham Singh Nagar District,Positive
407,Nufail Pachayi,2024-02-09 15:46:18.193526,5,0,0,0,Super!,Loved it,Thennala,Positive
406,Sunil,2024-02-09 15:46:18.193456,4,1,0,0,Wonderful,Worked fine on linux. Just plug and play. I te...,New Delhi,Positive
...,...,...,...,...,...,...,...,...,...,...
893,Praveen Kumar,2022-03-01 00:00:00.000000,5,0,0,0,Awesome,Using more than 6 months... No issues for now....,Erode,Positive
892,Mukesh Hendre,2022-03-01 00:00:00.000000,4,0,0,0,Delightful,Worth it....👌,Shirpur,Positive
891,Riya Santra,2022-03-01 00:00:00.000000,5,2,0,0,Just wow!,Value for money. In hand feels great the build...,Kolkata,Positive
890,Abhishek P P,2022-03-01 00:00:00.000000,5,0,0,0,Classy product,Loved it..,Thrissur District,Positive


In [46]:
#Preprocessing

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc) #remove any characters that are not alphanumeric or whitespace
    doc = doc.lower() #lowercase
    doc = doc.strip() #Removes leading and trailing whitespaces
    tokens = nltk.word_tokenize(doc) #Tokenizes the document into individual words
    filtered_tokens = [token for token in tokens if token not in stop_words] #Filters out stopwords
    doc = ' '.join(filtered_tokens) #Reconstructs the document from the filtered tokens by joining them back into a single string separated by spaces
    return doc

normalize_corpus = np.vectorize(normalize_document) #Vectorizes the normalize_document function

norm_corpus = normalize_corpus(list(most_recent_reviews['Review Text'])) # applying function to dataframe, producing a normalized corpus of documents
norm_corpus

array(['best product everyone', 'nice battery backup upto 8hr',
       'nice product value money', 'loved',
       'worked fine linux plug play tested steam rpcs3and heroic launcherworked three tested games days gone tekken 7 plastic great average size controller feels small depends person person think low connectivity issue 1mtr away pc connection lost gameplay connected itselfoverall good price first day review hope controller last long',
       'good product',
       'really bad product use 3 foot connection long means try 6 7 foot product single lost 3 4 foot signal correct many problems product please dont buy product',
       'good gaming tv company says work 10m radius true disconnected radius of6m good want gaming pad battery backup nice buy',
       'good controller',
       'long term review good worth money charge lasts long gaming sessions didnt face delay connectivity issues haptic good infact great price segment bought 14k comfortable hold long time one pression click iss

In [47]:
#convert a collection of text documents (norm_corpus) into a matrix of token counts

cv = CountVectorizer(ngram_range=(1, 2), min_df=20, max_df=0.8) #both single words and pairs of consecutive words. Tokens that occur in fewer than 20 documents will be ignored. Tokens that occur in more than 80% of the documents will be ignored.
cv_matrix = cv.fit_transform(norm_corpus) #transforms the corpus into a document-term matrix. The value at the intersection of a row and column represents the count of how many times that word or n-gram appears in the corresponding document.
cv_matrix.shape #(n_documents, n_features). n_documents represents the number of documents in the corpus, and n_features represents the number of unique words and n-grams in the vocabulary.

(550, 26)

In [48]:
sentiments = []
for text in norm_corpus:
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity
    if sentiment_score > 0:
        sentiments.append('Positive')
    elif sentiment_score < 0:
        sentiments.append('Negative')
    else:
        sentiments.append('Neutral')

# Add sentiment labels to DataFrame
most_recent_reviews['Sentiment'] = sentiments

# Display the DataFrame with sentiment labels
print(most_recent_reviews[['Review Text', 'Sentiment']])

                                           Review Text Sentiment
410                     Best product for everyone. 👍 👌  Positive
409              Nice, Battery backup is upto 8hr.....  Positive
408                       Nice product value for money  Positive
407                                           Loved it  Positive
406  Worked fine on linux. Just plug and play. I te...  Positive
..                                                 ...       ...
893  Using more than 6 months... No issues for now....  Positive
892                                      Worth it....👌  Positive
891  Value for money. In hand feels great the build...  Positive
890                                         Loved it..  Positive
888           excellent product. happy with this deal.  Positive

[550 rows x 2 columns]


/var/folders/9w/pgl8trgx1c5bwkrmm5hd5ktw0000gn/T/ipykernel_16247/196727771.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  most_recent_reviews['Sentiment'] = sentiments


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Name          2200 non-null   object        
 1   Date          2200 non-null   datetime64[ns]
 2   Rating        2200 non-null   int64         
 3   Images        2200 non-null   int64         
 4   Likes         2200 non-null   int64         
 5   Dislikes      2200 non-null   int64         
 6   Review Title  2200 non-null   object        
 7   Review Text   2200 non-null   object        
 8   location      2200 non-null   object        
 9   Sentiment     2200 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 172.0+ KB


In [50]:
sentiment_counts = most_recent_reviews['Sentiment'].value_counts()
sentiment_counts

Sentiment
Positive    460
Neutral      57
Negative     33
Name: count, dtype: int64

**Product Performance and Customer Satisfaction:**

The predominance of positive reviews among the most recent ones suggests that customers are generally satisfied with the product in recent times.
Implication: Businesses can interpret this as a positive indicator of product performance and customer satisfaction. It may indicate that recent improvements, updates, or enhancements have positively impacted the customer experience.

**Customer Engagement and Feedback Analysis:**

Monitoring recent reviews provides valuable insights into the evolving needs and preferences of customers.
Implication: Businesses should continue to engage with customers through review platforms, social media, and other channels to gather feedback and insights. Analyzing recent reviews allows businesses to identify emerging trends, address issues promptly, and capitalize on positive feedback to drive further improvements.

**Maintaining Quality and Consistency:**

The consistent presence of positive sentiment in recent reviews underscores the importance of maintaining product quality and consistency over time.
Implication: Businesses should remain vigilant in ensuring product quality, performance, and customer satisfaction. Consistent quality standards contribute to customer loyalty, positive word-of-mouth, and sustained business success.

**Addressing Negative Feedback:**

While the number of negative reviews is relatively low among the most recent ones, they still represent areas of concern or dissatisfaction.
Implication: Businesses should pay attention to negative feedback, even if it's in the minority. Addressing issues raised in negative reviews promptly and effectively demonstrates a commitment to customer satisfaction and can prevent potential escalations or reputational damage.

**Continuous Improvement and Innovation:**

The presence of both positive and negative sentiment in recent reviews highlights opportunities for continuous improvement and innovation.
Implication: Businesses should leverage insights from recent reviews to identify areas for enhancement or innovation. Investing in product development, feature updates, and customer support based on recent feedback can further enhance customer satisfaction and competitive advantage.

**Competitive Positioning:**

Analyzing sentiment trends in recent reviews relative to competitors can provide insights into competitive positioning and market dynamics.
Implication: Businesses should benchmark their product performance and customer sentiment against competitors to identify strengths, weaknesses, and opportunities for differentiation. Understanding competitive positioning helps businesses refine their strategies and offerings to stay ahead in the market.

In [51]:
# Filter positive sentiment reviews
recent_positive_reviews_df = most_recent_reviews[most_recent_reviews['Sentiment'] == 'Positive']

In [52]:
# Tokenize and preprocess positive sentiment reviews
recent_positive_norm_corpus = normalize_corpus(recent_positive_reviews_df['Review Text'])

In [53]:
# Vectorize text using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000, max_df=0.8, min_df=20)
tfidf_matrix = tfidf_vectorizer.fit_transform(recent_positive_norm_corpus)

In [54]:
# Apply Latent Dirichlet Allocation (LDA) for topic modeling
num_topics = 7  # You can adjust the number of topics as needed
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_topic_matrix = lda_model.fit_transform(tfidf_matrix)

In [55]:
# Display topics and associated top words
feature_names = tfidf_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda_model.components_):
    print(f"Topic {topic_idx+1}:")
    top_words_idx = topic.argsort()[:-10 - 1:-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(top_words)

Topic 1:
['best', 'gamepad', 'price', 'good', 'product', 'using', 'quality', 'awesome', 'games', 'controller']
Topic 2:
['working', 'gaming', 'play', 'good', 'games', 'using', 'price', 'also', 'product', 'nice']
Topic 3:
['good', 'product', 'quality', 'price', 'best', 'games', 'build', 'using', 'nice', 'gamepad']
Topic 4:
['controller', 'great', 'price', 'using', 'one', 'quality', 'games', 'best', 'good', 'awesome']
Topic 5:
['nice', 'product', 'games', 'quality', 'using', 'good', 'awesome', 'gamepad', 'controller', 'backup']
Topic 6:
['works', 'one', 'also', 'product', 'awesome', 'good', 'using', 'gaming', 'backup', 'price']
Topic 7:
['awesome', 'battery', 'build', 'good', 'backup', 'quality', 'games', 'also', 'using', 'controller']


**Topic 1: Best Value and Quality:**

Terms: 'best', 'gamepad', 'price', 'good', 'product', 'quality', 'awesome', 'games', 'controller'
Interpretation: This topic emphasizes the product's value proposition, highlighting its quality, affordability, and versatility for gaming.

**Topic 2: Performance and Gaming Experience:**

Terms: 'working', 'gaming', 'play', 'good', 'games', 'using', 'price', 'also', 'product', 'nice'
Interpretation: This topic focuses on the product's performance, functionality, and overall gaming experience, indicating satisfaction with its usability and effectiveness.

**Topic 3: Quality and Value Proposition:**

Terms: 'good', 'product', 'quality', 'price', 'best', 'games', 'build', 'using', 'nice', 'gamepad'
Interpretation: Similar to Topic 1, this topic underscores the product's quality and value proposition, including its build quality, affordability, and suitability for gaming.

**Topic 4: Controller Performance and Satisfaction:**

Terms: 'controller', 'great', 'price', 'using', 'one', 'quality', 'games', 'best', 'good', 'awesome'
Interpretation: This topic highlights customer satisfaction with the product's controller functionality, performance, and value for money.

**Topic 5: Quality and Usability:**

Terms: 'nice', 'product', 'games', 'quality', 'using', 'good', 'awesome', 'gamepad', 'controller', 'backup'
Interpretation: This topic emphasizes the product's overall quality, usability, and reliability, including its performance across different gaming scenarios.

**Topic 6: Versatility and Performance:**

Terms: 'works', 'one', 'also', 'product', 'awesome', 'good', 'using', 'gaming', 'backup', 'price'
Interpretation: This topic highlights the product's versatility, effectiveness, and value for gaming enthusiasts, indicating satisfaction with its performance and features.

**Topic 7: Battery Backup and Quality:**

Terms: 'awesome', 'battery', 'build', 'good', 'backup', 'quality', 'games', 'also', 'using', 'controller'
Interpretation: This topic emphasizes the product's battery backup, build quality, and overall gaming experience, suggesting satisfaction with its durability and reliability.


**Product Development and Enhancement:**

The identified topics provide insights into the features and aspects of the product that customers value most.
Implication: Businesses can use this information to prioritize product development efforts, focusing on enhancing key features such as performance, build quality, and battery backup to meet customer expectations effectively.

**Marketing and Messaging:**

Understanding the themes and sentiments expressed in positive reviews enables businesses to craft targeted marketing messages.
Implication: Businesses can leverage the identified topics to emphasize the product's value proposition, quality, and performance in marketing campaigns, attracting new customers and reinforcing brand loyalty among existing ones.

**Customer Support and Engagement:**

Positive reviews reflect customer satisfaction, but businesses should continue to engage with customers and address any concerns promptly.
Implication: Providing responsive customer support, gathering feedback, and maintaining open communication channels with customers are essential for sustaining positive sentiment and fostering long-term relationships.

**Competitive Analysis and Differentiation:**

Analyzing positive reviews relative to competitors' offerings can provide insights into areas of competitive advantage and differentiation.
Implication: Businesses should benchmark their product against competitors, identifying unique selling points and opportunities for differentiation to stand out in the market and attract customers.

**Continuous Monitoring and Improvement:**

Monitoring sentiment trends and feedback allows businesses to identify emerging trends and areas for improvement.
Implication: Continuously monitoring customer feedback and sentiment enables businesses to adapt quickly, address issues, and drive continuous improvement in product quality, performance, and customer satisfaction.

In [56]:
# Perform KMeans clustering
num_clusters = 3  # You can adjust the number of clusters as needed
kmeans_model = KMeans(n_clusters=num_clusters, random_state=42)
kmeans_clusters = kmeans_model.fit_predict(tfidf_matrix)

/Users/bijitgoswami/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [57]:
# Add cluster labels to the DataFrame
recent_positive_reviews_df['Cluster'] = kmeans_clusters

/var/folders/9w/pgl8trgx1c5bwkrmm5hd5ktw0000gn/T/ipykernel_16247/4064974394.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_positive_reviews_df['Cluster'] = kmeans_clusters


In [58]:
# Display the DataFrame with cluster labels
print(recent_positive_reviews_df[['Review Text', 'Cluster']])

                                           Review Text  Cluster
410                     Best product for everyone. 👍 👌        1
409              Nice, Battery backup is upto 8hr.....        2
408                       Nice product value for money        2
407                                           Loved it        1
406  Worked fine on linux. Just plug and play. I te...        1
..                                                 ...      ...
893  Using more than 6 months... No issues for now....        1
892                                      Worth it....👌        1
891  Value for money. In hand feels great the build...        1
890                                         Loved it..        1
888           excellent product. happy with this deal.        2

[460 rows x 2 columns]


In [59]:
recent_negative_reviews_df = most_recent_reviews[most_recent_reviews['Sentiment'] == 'Negative']

In [60]:
recent_negative_reviews_df

,Name,Date,Rating,Images,Likes,Dislikes,Review Title,Review Text,location,Sentiment
404,Flipkart Customer,2024-02-09 15:46:18.193313,1,0,0,0,Worst experience ever!,Really bad product I will use 3 foot only the...,Dahanu,Negative
424,Flipkart Customer,2024-01-09 15:46:18.194718,1,0,0,0,Very poor,Not working in game but on pc screen,New Delhi,Negative
411,Arshan S,2024-01-09 15:46:18.193810,1,0,0,0,Unsatisfactory,Worst products because many game not acceptabl...,Chennai,Negative
438,tabish mallick,2023-12-09 15:46:18.195708,1,0,0,0,Unsatisfactory,Forest remote,Kolkata,Negative
462,RANJAY PATEL,2023-11-09 15:46:18.197379,2,0,0,0,Moderate,Bad this pro,Katihar District,Negative
249,Jayanth IndLa,2023-11-09 15:46:18.169781,2,0,1,0,Slightly disappointed,"I did not understand, quality is poor compared...",Bengaluru,Negative
489,Raj Kar,2023-09-09 15:46:18.199256,1,0,0,0,Not recommended at all,worst controller .. quality is not good and fr...,Silchar,Negative
250,SK KHALID SAIFULLAH,2023-09-09 15:46:18.169865,2,0,1,0,Moderate,Over priced. Very complex design. Very compli...,Kolkata,Negative
511,Ivan Ehsan,2023-07-09 15:46:18.200802,1,0,0,0,Did not meet expectations,Dont buy this ... worst quality I have seen no...,Jodhpur District,Negative
532,Ashis Kumar Majhi,2023-05-09 15:46:18.202300,1,0,0,0,Worthless,"I was having some issues with the gamepad, so ...",New Town,Negative


In [61]:
recent_negative_reviews_df['Sentiment'].value_counts()

Sentiment
Negative    33
Name: count, dtype: int64

In [62]:
# Tokenize and preprocess positive sentiment reviews
recent_negative_norm_corpus = normalize_corpus(recent_negative_reviews_df['Review Text'])

In [63]:
# Vectorize text using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000, max_df=0.8, min_df=5)
tfidf_matrix = tfidf_vectorizer.fit_transform(recent_negative_norm_corpus)

In [64]:
tfidf_matrix

<33x7 sparse matrix of type '<class 'numpy.float64'>'
	with 40 stored elements in Compressed Sparse Row format>

In [65]:
# Apply Latent Dirichlet Allocation (LDA) for topic modeling
num_topics = 3  # You can adjust the number of topics as needed
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_topic_matrix = lda_model.fit_transform(tfidf_matrix)

In [66]:
# Display topics and associated top words
feature_names = tfidf_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda_model.components_):
    print(f"Topic {topic_idx+1}:")
    top_words_idx = topic.argsort()[:-10 - 1:-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(top_words)

Topic 1:
['product', 'work', 'bad', 'dont', 'quality', 'working', 'worst']
Topic 2:
['working', 'dont', 'bad', 'quality', 'work', 'product', 'worst']
Topic 3:
['worst', 'quality', 'work', 'dont', 'working', 'bad', 'product']


**Topic 1: Product Quality and Functionality:**

Terms: 'product', 'work', 'bad', 'dont', 'quality', 'working', 'worst'
Interpretation: This topic emphasizes dissatisfaction with the product's quality, functionality, and overall performance, indicating a negative customer experience.

**Topic 2: Product Performance and Functionality:**

Terms: 'working', 'dont', 'bad', 'quality', 'work', 'product', 'worst'
Interpretation: Similar to Topic 1, this topic focuses on issues related to the product's performance, functionality, and quality, highlighting customer dissatisfaction.

**Topic 3: Overall Dissatisfaction:**

Terms: 'worst', 'quality', 'work', 'dont', 'working', 'bad', 'product'
Interpretation: This topic reinforces the sentiment of overall dissatisfaction with the product, emphasizing its poor quality, functionality, and performance.


**Quality Assurance and Improvement:**

The consistent emphasis on product quality and functionality in negative reviews underscores the importance of quality assurance and improvement efforts.
Implication: Businesses should prioritize identifying and addressing issues related to product quality, functionality, and performance through rigorous testing, quality control measures, and continuous improvement processes.

**Customer Support and Issue Resolution:**

Negative reviews highlight areas where customers have encountered issues or dissatisfaction with the product.
Implication: Providing prompt and effective customer support, addressing customer concerns, and offering solutions or replacements can help mitigate negative sentiment and rebuild customer trust.

**Product Development and Innovation:**

Insights from negative reviews can inform product development and innovation efforts to address customer needs and preferences.
Implication: Businesses should leverage feedback from negative reviews to identify opportunities for product enhancement, feature updates, and innovation, ensuring that future iterations meet or exceed customer expectations.

**Transparency and Communication:**

Transparent communication about identified issues, planned resolutions, and improvements is essential for managing customer expectations and rebuilding trust.
Implication: Businesses should communicate openly with customers affected by negative experiences, providing updates and assurances regarding resolution efforts and improvements to demonstrate their commitment to customer satisfaction.

**Competitive Analysis and Benchmarking:**

Analyzing negative reviews relative to competitors' products can provide insights into areas of competitive advantage or weakness.
Implication: Benchmarking the product against competitors' offerings and addressing identified shortcomings can help businesses differentiate themselves in the market and improve overall customer satisfaction.

In [67]:
# Perform KMeans clustering
num_clusters = 3  # You can adjust the number of clusters as needed
kmeans_model = KMeans(n_clusters=num_clusters, random_state=42)
kmeans_clusters = kmeans_model.fit_predict(tfidf_matrix)

/Users/bijitgoswami/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [68]:
# Add cluster labels to the DataFrame
recent_negative_reviews_df['Cluster'] = kmeans_clusters

/var/folders/9w/pgl8trgx1c5bwkrmm5hd5ktw0000gn/T/ipykernel_16247/925747853.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_negative_reviews_df['Cluster'] = kmeans_clusters


In [69]:
# Display the DataFrame with cluster labels
print(recent_negative_reviews_df[['Review Text', 'Cluster']])

                                           Review Text  Cluster
404  Really bad product I will use 3 foot  only the...        2
424               Not working in game but on pc screen        1
411  Worst products because many game not acceptabl...        1
438                                      Forest remote        0
462                                       Bad this pro        2
249  I did not understand, quality is poor compared...        0
489  worst controller .. quality is not good and fr...        0
250  Over priced. Very complex design.  Very compli...        0
511  Dont buy this ... worst quality I have seen no...        0
532  I was having some issues with the gamepad, so ...        0
546                                      Bast game pad        0
558  The product work fine for few months and they ...        2
567                                 Average controller        0
571  Been using for more than a month. Absolutely w...        1
617                     Bad product Miss

# Phase 3

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Name          2200 non-null   object        
 1   Date          2200 non-null   datetime64[ns]
 2   Rating        2200 non-null   int64         
 3   Images        2200 non-null   int64         
 4   Likes         2200 non-null   int64         
 5   Dislikes      2200 non-null   int64         
 6   Review Title  2200 non-null   object        
 7   Review Text   2200 non-null   object        
 8   location      2200 non-null   object        
 9   Sentiment     2200 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 172.0+ KB


In [71]:
df.head(20)

,Name,Date,Rating,Images,Likes,Dislikes,Review Title,Review Text,location,Sentiment
0,soumyadeep sharma,2019-02-01 00:00:00.000000,5,5,518,78,Highly recommended,great product ... love it ... my favorite game...,North Twenty Four Parganas District,Positive
1,Krishna Rao,2016-08-01 00:00:00.000000,5,0,106,15,Coolest piece ever had,"Very good product, must have for gamers.",Bangalore,Positive
2,Chanchal Pan,2018-12-01 00:00:00.000000,5,1,44,4,Great product,best,Burdwan,Positive
3,abhishek kumar,2016-06-01 00:00:00.000000,5,0,248,49,LIKE XBOX 360,At first time i got a defective piece it's rec...,Patna,Positive
4,YUSUF RASOOL KHAN,2021-07-01 00:00:00.000000,5,1,27,2,Super!,"Very nice game pet, runs very smooth thank you...",Nagpur,Positive
5,Reena Rajput,2019-02-01 00:00:00.000000,5,1,12,0,Super!,Awesome 👌,Jammu,Positive
6,Flipkart Customer,2023-04-09 15:46:18.121780,5,1,18,1,Terrific,Amazing product I love it vibration is very go...,Rohtak,Positive
7,Flipkart Customer,2021-10-01 00:00:00.000000,5,0,17,1,Excellent,Yes. It works with Android TV and I know that ...,New Delhi,Positive
8,taha Adnan,2018-12-01 00:00:00.000000,5,1,117,26,Terrific,Better than many other pricy gamepad..... Grea...,Warangal,Positive
9,Nilesh Talekar,2020-05-01 00:00:00.000000,5,1,17,1,Mind-blowing purchase,i've been using this controller since 2 yearsi...,Aurangabad,Positive


In [72]:
# Preprocess and vectorize the review text
norm_corpus = normalize_corpus(df['Review Text'])
tfidf_vectorizer = TfidfVectorizer(max_features=1000, max_df=0.8, min_df=5)
tfidf_matrix = tfidf_vectorizer.fit_transform(norm_corpus)

In [73]:
target_map = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
df['Sentiment'] = df['Sentiment'].map(target_map)

In [74]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df['Sentiment'], test_size=0.2, random_state=42)

In [75]:
print(df['Sentiment'].unique())

[ 1  0 -1]


In [76]:
df.head()

,Name,Date,Rating,Images,Likes,Dislikes,Review Title,Review Text,location,Sentiment
0,soumyadeep sharma,2019-02-01,5,5,518,78,Highly recommended,great product ... love it ... my favorite game...,North Twenty Four Parganas District,1
1,Krishna Rao,2016-08-01,5,0,106,15,Coolest piece ever had,"Very good product, must have for gamers.",Bangalore,1
2,Chanchal Pan,2018-12-01,5,1,44,4,Great product,best,Burdwan,1
3,abhishek kumar,2016-06-01,5,0,248,49,LIKE XBOX 360,At first time i got a defective piece it's rec...,Patna,1
4,YUSUF RASOOL KHAN,2021-07-01,5,1,27,2,Super!,"Very nice game pet, runs very smooth thank you...",Nagpur,1


In [77]:
print("Number of values in X_train:", X_train.shape[0])
print("Number of values in X_test:", X_test.shape[0])
print("Number of values in y_train:", y_train.shape[0])
print("Number of values in y_test:", y_test.shape[0])

Number of values in X_train: 1760
Number of values in X_test: 440
Number of values in y_train: 1760
Number of values in y_test: 440


In [78]:
# Train a Logistic Regression classifier
classifier = LogisticRegression(random_state=42)
classifier.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [79]:
# Predictions on the test set
y_pred = classifier.predict(X_test)

In [80]:
# Evaluate the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       1.00      0.23      0.37        22
           0       0.76      0.35      0.47        55
           1       0.88      0.99      0.93       363

    accuracy                           0.88       440
   macro avg       0.88      0.52      0.59       440
weighted avg       0.87      0.88      0.85       440



The results of the Logistic Regression classifier for identifying positive or negative reviews are as follows:

- **Precision**: Precision measures the proportion of correctly predicted positive or negative reviews out of all predicted positive or negative reviews. 
  - For the negative class (-1), precision is 1.00, indicating that when the classifier predicts a review as negative, it is almost always correct.
  - For the neutral class (0), precision is 0.76, suggesting that there's a moderate level of accuracy in predicting neutral reviews.
  - For the positive class (1), precision is 0.88, indicating a high level of accuracy in predicting positive reviews.

- **Recall**: Recall measures the proportion of correctly predicted positive or negative reviews out of all actual positive or negative reviews.
  - For the negative class (-1), recall is 0.23, indicating that the classifier correctly identifies only a small portion of actual negative reviews.
  - For the neutral class (0), recall is 0.35, suggesting that the classifier correctly identifies only a moderate portion of actual neutral reviews.
  - For the positive class (1), recall is 0.99, indicating that the classifier correctly identifies almost all actual positive reviews.

- **F1-score**: The F1-score is the harmonic mean of precision and recall and provides a balance between the two metrics.
  - F1-score for the negative class (-1) is 0.37, for the neutral class (0) is 0.47, and for the positive class (1) is 0.93.

- **Accuracy**: Accuracy measures the overall correctness of predictions across all classes.
  - The overall accuracy of the classifier is 0.88, suggesting that it correctly predicts the sentiment of reviews in the dataset 88% of the time.

**Interpretation and Business Implications**:

1. **Class Imbalance**: The classifier performs well in predicting positive reviews (class 1) with high precision, recall, and F1-score. However, it struggles with predicting negative reviews (class -1) and to a lesser extent with neutral reviews (class 0) due to class imbalance.

2. **Improving Negative Sentiment Detection**: To improve the detection of negative sentiment, the classifier may benefit from strategies such as:
   - Collecting more labeled data for negative reviews to balance class distribution.
   - Exploring techniques like oversampling, undersampling, or generating synthetic samples for the minority class.
   - Fine-tuning the classifier parameters or experimenting with different algorithms that handle class imbalance better.

3. **Business Implications**:
   - The classifier's high precision for positive reviews indicates its reliability in identifying satisfied customers. This can be valuable for businesses in gauging customer satisfaction levels and identifying areas of strength.
   - However, the lower recall for negative reviews suggests that the classifier may miss some instances of dissatisfaction or negative sentiment. This could lead to missed opportunities for addressing customer concerns and improving overall customer experience.
   - Businesses should consider using the classifier results as one of multiple inputs in their decision-making processes. Human review and interpretation may still be necessary, especially for negative reviews, to ensure all customer feedback is appropriately addressed.
   - Continuous monitoring and refinement of the classifier based on feedback and performance metrics can lead to more accurate sentiment analysis and better-informed business decisions over time.

In [81]:
from sklearn.metrics import accuracy_score

# Assuming y_test and y_pred are the true labels and predicted labels, respectively
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.875


An accuracy of 0.875 means that the classifier correctly predicts the sentiment of reviews approximately 87.5% of the time. This indicates a relatively high level of overall correctness in predicting whether a review is positive or negative.

**Interpretation:**

High Overall Performance: An accuracy of 0.875 suggests that the classifier performs well in distinguishing between positive and negative reviews in the dataset.

Reliability: With a high accuracy score, the classifier can be considered reliable in predicting the sentiment of reviews, which can be valuable for businesses in understanding customer feedback and sentiments.

Confidence in Predictions: The high accuracy score indicates that the classifier's predictions can be trusted with a high level of confidence, which can assist businesses in making informed decisions based on sentiment analysis.

**Business Implications:**

Effective Customer Feedback Analysis: The classifier's high accuracy enables businesses to effectively analyze customer feedback and sentiments, helping them understand customer preferences, satisfaction levels, and areas for improvement.

Decision Support: Reliable sentiment analysis can serve as a valuable decision support tool for businesses, guiding strategies related to product development, marketing campaigns, customer service enhancements, and overall business operations.

Identifying Trends and Patterns: By accurately predicting sentiment, businesses can identify emerging trends, patterns, and issues within customer feedback, allowing them to proactively address concerns and capitalize on opportunities.

Enhancing Customer Experience: Accurate sentiment analysis enables businesses to tailor their products, services, and communication strategies to better meet customer needs and expectations, ultimately enhancing the overall customer experience.

Reputation Management: Effective sentiment analysis can help businesses manage their online reputation by identifying and addressing negative sentiments promptly, thereby mitigating potential reputational risks and fostering positive brand perception.

Overall, an accuracy score of 0.875 signifies the classifier's effectiveness in predicting sentiment, providing businesses with valuable insights to drive informed decision-making and improve customer satisfaction and loyalty.

In [82]:
conf_matrix = confusion_matrix(y_test, y_pred)

conf_matrix

array([[  5,   4,  13],
       [  0,  19,  36],
       [  0,   2, 361]])

The confusion matrix provided indicates the following counts for each predicted class against their actual classes:

- For the class labeled as 0 (which presumably represents negative sentiment):
  - True Negatives (predicted correctly as negative): 5
  - False Positives (predicted as negative but actually positive): 4
  - False Negatives (predicted as positive but actually negative): 13

- For the class labeled as 1 (which presumably represents neutral sentiment):
  - True Positives (predicted correctly as neutral): 19
  - False Negatives (predicted as negative but actually neutral): 0
  - False Positives (predicted as neutral but actually positive): 36

- For the class labeled as 2 (which presumably represents positive sentiment):
  - True Positives (predicted correctly as positive): 361
  - False Negatives (predicted as negative but actually positive): 2
  - False Positives (predicted as positive but actually neutral): 0

**Interpretation**:

1. The confusion matrix provides a detailed breakdown of the classifier's performance across different sentiment classes.

2. It highlights the instances where the classifier made correct predictions (true positives and true negatives) as well as where it misclassified instances (false positives and false negatives).

**Business Implications**:

1. **Identifying Weaknesses**: The confusion matrix helps identify which sentiment classes the classifier struggles with. For example, it appears to have more difficulty in accurately predicting negative and neutral sentiments compared to positive sentiments.

2. **Improving Performance**: Understanding where misclassifications occur allows for targeted improvements in the classifier's performance. For instance, strategies can be devised to address misclassifications in negative and neutral sentiments, such as collecting more labeled data, adjusting classifier parameters, or exploring different algorithms.

3. **Resource Allocation**: The confusion matrix assists in allocating resources effectively. For instance, if misclassifications in certain sentiment classes are deemed more detrimental to business outcomes, resources can be prioritized for improving the classifier's performance in those classes.

4. **Enhancing Decision-Making**: A clear understanding of the classifier's performance enables more informed decision-making. Businesses can use this information to determine the reliability of sentiment predictions and adjust strategies accordingly, such as adjusting marketing campaigns, refining product features, or enhancing customer service efforts based on sentiment analysis results.

# PHASE 4

In [98]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from datetime import date
from sklearn.ensemble import RandomForestClassifier
# Combine features
from scipy.sparse import hstack


# Load the dataset
#data = pd.read_csv("your_data.csv")

### Preprocess the review text
#def preprocess_text(text):
  # Clean text (lowercase, remove stop words, punctuation)
  # ...
#  return cleaned_text

df["Review Text"] = norm_corpus #data["Review Text"].apply(preprocess_text)


# Extract features
df["Text Length"] = df["Review Text"].str.len()
df["Unique Words"] = df["Review Text"].apply(lambda text: len(set(text.split())))
df["Has Image"] = df["Images"].apply(lambda x: 1 if x > 0 else 0)


# Calculate time elapsed since review
today = date.today()
today_datetime = datetime.combine(today, datetime.min.time())
df["Time Elapsed"] = (today_datetime - pd.to_datetime(df["Date"])) / np.timedelta64(1, 'D')

# Feature engineering with likes/dislikes ratio
df["Likes Ratio"] = df["Likes"] / (df["Likes"] + df["Dislikes"] + 1)  # Avoid division by zero

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[["Review Text","Text Length", "Unique Words", "Has Image", "Sentiment", "Time Elapsed", "Likes Ratio"]], df["Likes"], test_size=0.2)

# Feature vectorization (convert text to numerical features)
vectorizer = TfidfVectorizer(max_features=1000)
X_train_text = vectorizer.fit_transform(X_train["Review Text"])
X_test_text = vectorizer.transform(X_test["Review Text"])



# Concatenate sparse matrices
X_train_combined = hstack((X_train_text, X_train.drop("Review Text", axis=1)))
X_test_combined = hstack((X_test_text, X_test.drop("Review Text", axis=1)))
#X_train_combined = pd.concat([X_train_text, X_train.drop("Review Text", axis=1)], axis=1)
#X_test_combined = pd.concat([X_test_text, X_test.drop("Review Text", axis=1)], axis=1)

# Train a model (Logistic Regression in this example)
model = RandomForestClassifier()
model.fit(X_train_combined, y_train)

# Evaluate the model (replace with desired metrics)
predictions = model.predict(X_test_combined)
accuracy = accuracy_score(y_test, predictions)  # If predicting categorical values
print(f"Model Accuracy: {accuracy}")

# Feature importance
importances = model.feature_importances_
feature_names = list(vectorizer.get_feature_names_out()) + ["Text Length", "Unique Words", "Has Image", "Sentiment", "Time Elapsed", "Likes Ratio"]

# Zip feature names and importances and sort them based on importances
feature_importance_list = list(zip(feature_names, importances))
feature_importance_list.sort(key=lambda x: x[1], reverse=True)

# Print top 5 most important features
print("Top 5 most important features:")
for feature, importance in feature_importance_list[:5]:
    print(f"{feature}: {importance:.4f}")


Model Accuracy: 0.9340909090909091
Top 5 most important features:
Likes Ratio: 0.2899
Time Elapsed: 0.0385
Has Image: 0.0341
Text Length: 0.0271
Unique Words: 0.0209


Report 4: Helpfulness Predictive Model

We have used RandomForestClassifier as the model.

Top Factors in Helpfulness Prediction:

Based on the feature importances (Likes Ratio: 0.2899, Time Elapsed: 0.0385, Has Image: 0.0341, Text Length: 0.0271, Unique Words: 0.0209), the following factors play the most significant roles in helpfulness prediction:

Likes Ratio (0.2899): This feature holds the highest importance, suggesting that the model heavily relies on the ratio of positive ("Likes") to negative ("Dislikes") feedback to predict helpfulness. A high ratio indicates a generally well-received review, likely perceived as more helpful.

Time Elapsed (0.0385): While less impactful than the Likes Ratio, this feature still contributes, suggesting that reviews tend to gain helpfulness votes over time. This could be because valuable information takes time to be recognized or because newer reviews might not have been seen by many users yet.

Other Contributing Factors:

Has Image (0.0341): The presence of an image has a moderate impact. 

Text Length (0.0271) & Unique Words (0.0209): These features have a lower influence but still contribute. Longer reviews with a wider vocabulary might convey more details and potentially be perceived as more informative.

Use of Images in Review:
Here in this case the presence of image had a moderate effect.
Positive Impact: Images can be helpful if they:

Visually demonstrate product features or usage.
Showcase the product's condition (especially for used items).
Enhance understanding of the review content.
Negative Impact: Images might not be helpful if they are:

Irrelevant to the review's focus.
Poor quality or unclear.